In [3]:
### This will generate a yaml file for each location ID
import girder_client
import os, io, yaml

gc = girder_client.GirderClient(apiUrl='https://styx.neurology.emory.edu/girder/api/v1')
_ = gc.authenticate(interactive=True)


In [4]:
tmaDataSetMaps=[{'slideName':'254461.svs','stainId':"HE", 'itemId': '62ded6e6e70def4ae726b22a','tmaAnnotationId':'62e58f50e70def4ae7272db2'},
                   {'slideName':'257593.svs', 'stainId': "LINK48",'itemId':'62ded6e7e70def4ae726b407','tmaAnnotationId':'62e3273de70def4ae726e8dc'},
               {'slideName':"257957.svs", "stainId":"LEICA","itemId":"62ded6e7e70def4ae726b40f","tmaAnnotationId":"62e32812e70def4ae726e9a1"}]

In [6]:
sampleAnnotationId="62e08784e70def4ae726d0a6"

a = gc.get(f'annotation/{sampleAnnotationId}')

tmaCoreElemenets =a.get('annotation',{}).get('elements',[])


In [5]:
## I already have the tmeCores in this folder:
allCoreItems = list(gc.listItem('62e310b4e70def4ae726d5d9'))

print(len(allCoreItems),"cores detected")



1185 cores detected


In [11]:
## Want this organized by slideNum and then coreId..

coreItemDict = {}
for coreItem in allCoreItems:
    nameParts = coreItem['name'].split('.')
    slideNum = nameParts[0]
    coreId = nameParts[1]

    if slideNum not in coreItemDict:
        coreItemDict[slideNum] = {}

    coreItemDict[slideNum][coreId] = coreItem



dict_keys(['S1', 'S11', 'S14', 'S15', 'S17', 'S18', 'S2', 'S5', 'S7', 'S9'])


In [12]:
## Now want to create a yaml file for the cores I am interested in.. which is all of them!  So I can get the coreIds and the n go from there
slideSet_for_core_combo =[{'slideNum':'S18','stainPlatform':'HE'},{'slideNum':'S7','stainPlatform':'LINK48'},{'slideNum':'S11','stainPlatform':'LEICA'},{'slideNum':'S15','stainPlatform':"Ventana"},{'slideNum': 'S1','stainPlatform':'ONICS'}]

## TMA S18 is the HE image, so will use this for reference
tmaCoreId_list = coreItemDict['S18'].keys()



dict_keys(['A1', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'C1', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D1', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'F1', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'G1', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'H1', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'I1', 'I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'J1', 'J10', 'J11', 'J12', 'J13', 'J14', 'J15', 'J2', 'J3', 'J4', 'J5', 'J6', 'J7', 'J8', 'J9'])


In [23]:

def generate_yaml_object(core_id):
    sources = []
    # Iterate through each slide set
    for idx, slideSet in enumerate(slideSet_for_core_combo):
        slideNum = slideSet['slideNum']
        
        # Skip if the core doesn't exist in this slide
        if slideNum not in coreItemDict or core_id not in coreItemDict[slideNum]:
            continue
            
        # Get the item ID for this core
        item_id = coreItemDict[slideNum][core_id]['_id']
        
        # Create source entry
        source = {
            'path': f'girder://{item_id}',
            'z': 0,
            'position': {
                'x': idx * 3072,  # Offset each image by 3072
                'y': 0
            }
        }
        sources.append(source)
    
    # Return YAML content as dictionary
    return {'sources': sources, 'backgroundColor': [255,255,255]}

In [24]:
# yaml_obj=   generate_yaml_object('A1')
# yaml_string = yaml.dump(yaml_obj, default_flow_style=False)

# print(yaml_string)

In [25]:

compositeFolderId = '67901080132e68fc06fbdf82'

def upload_yaml_to_dsa(yaml_string, filename, folder_id, gc):
    # Convert string to bytes-like object
    yaml_bytes = yaml_string.encode('utf-8')
    
    # Create a file-like object
    file_obj = io.BytesIO(yaml_bytes)
    
    # Upload to DSA/Girder
    gc.uploadFile(
        folder_id,
        file_obj,
        filename,
        len(yaml_string),
        parentType='folder'
    )
    


In [26]:
for coreId in tmaCoreId_list:
    print(f"Generating yaml for {coreId}")
    yaml_obj = generate_yaml_object(coreId)
    yaml_string = yaml.dump(yaml_obj, default_flow_style=False)
    upload_yaml_to_dsa(yaml_string, f'{coreId}.yaml', compositeFolderId, gc)



Generating yaml for A1
Generating yaml for A10
Generating yaml for A11
Generating yaml for A12
Generating yaml for A13
Generating yaml for A14
Generating yaml for A15
Generating yaml for A2
Generating yaml for A3
Generating yaml for A4
Generating yaml for A5
Generating yaml for A6
Generating yaml for A7
Generating yaml for A8
Generating yaml for A9
Generating yaml for B1
Generating yaml for B10
Generating yaml for B11
Generating yaml for B12
Generating yaml for B13
Generating yaml for B14
Generating yaml for B15
Generating yaml for B2
Generating yaml for B3
Generating yaml for B4
Generating yaml for B5
Generating yaml for B6
Generating yaml for B7
Generating yaml for B8
Generating yaml for B9
Generating yaml for C1
Generating yaml for C10
Generating yaml for C11
Generating yaml for C12
Generating yaml for C13
Generating yaml for C14
Generating yaml for C15
Generating yaml for C2
Generating yaml for C3
Generating yaml for C4
Generating yaml for C5
Generating yaml for C6
Generating yaml 

In [27]:
from PIL import Image, ImageDraw, ImageFont
import math
import io
import requests

def fetch_dsa_image(gc, item_id):
    """Fetch image from DSA and convert to PIL Image"""
    # Get thumbnail URL (or full image URL depending on your needs)
    url = f"{gc.urlBase}/item/{item_id}/tiles/thumbnail"
    
    # Use authentication from girder client
    response = requests.get(url, headers={'Girder-Token': gc.token})
    
    # Convert response content to PIL Image
    image_bytes = io.BytesIO(response.content)
    return Image.open(image_bytes)

# def create_dsa_image_grid(gc, item_list, output_size=(1600, 1600)):
#     """
#     Creates a grid from DSA images
    
#     Args:
#         gc: Authenticated girder client
#         item_list: List of DSA items containing image info
#         output_size: Desired size of final image
#     """
#     images = []
#     titles = []
    
#     # Fetch all images and prepare titles
#     for item in item_list:
#         try:
#             img = fetch_dsa_image(gc, item['_id'])
#             images.append(img)
#             # Assuming item name format: "slide.coreId.x.y.png"
#             titles.append(item['name'].split('.')[0:2][0])  # Just get slide.coreId
#         except Exception as e:
#             print(f"Error fetching image for {item['name']}: {e}")
#             continue

#     # Calculate grid dimensions
#     n_images = len(images)
#     grid_size = math.ceil(math.sqrt(n_images))
    
#     # Calculate individual cell size
#     cell_width = output_size[0] // grid_size
#     cell_height = (output_size[1] // grid_size) + 40  # 40 pixels for text
    
#     # Create blank white image
#     background = Image.new('RGB', 
#                          (cell_width * grid_size, cell_height * grid_size), 
#                          'white')
#     draw = ImageDraw.Draw(background)
    
#     # Try to load a font
#     try:
#         font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 14)
#     except:
#         font = ImageFont.load_default()

#     # Place each image and its title
#     for idx, (img, title) in enumerate(zip(images, titles)):
#         # Calculate position
#         row = idx // grid_size
#         col = idx % grid_size
        
#         # Resize image to fit in cell (leaving room for text)
#         img_width = cell_width - 20  # 10px padding on each side
#         img_height = cell_height - 50  # Leave room for text
#         img = img.resize((img_width, img_height), Image.Resampling.LANCZOS)
        
#         # Calculate positions
#         x = col * cell_width + 10  # 10px padding
#         y = row * cell_height + 35  # Leave room for text
        
#         # Paste image
#         background.paste(img, (x, y))
        
#         # Add title
#         text_x = col * cell_width + (cell_width // 2)
#         text_y = row * cell_height + 10
#         draw.text((text_x, text_y), title, 
#                  fill='black', 
#                  font=font, 
#                  anchor="mm")  # middle middle alignment

#     return background



"\n# Assuming you have a list of DSA items\ngrid_image = create_dsa_image_grid(gc, your_item_list)\ngrid_image.save('tma_grid.png')\n\n# Or to upload back to DSA:\nbuffer = io.BytesIO()\ngrid_image.save(buffer, format='PNG')\nbuffer.seek(0)\ngc.uploadFile(\n    folder_id,\n    buffer,\n    'tma_grid.png',\n    size=buffer.getbuffer().nbytes,\n    parentType='folder'\n)\n"

In [42]:
# def create_dsa_image_grid(gc, item_list, output_size=(1600, 1600)):
#     """
#     Creates a grid from DSA images while maintaining aspect ratio
#     """
#     images = []
#     titles = []
    
#     # Fetch all images and prepare titles
#     for item in item_list:
#         try:
#             img = fetch_dsa_image(gc, item['_id'])
#             images.append(img)
#             titles.append(item['name'].split('.')[0:2][0])
#         except Exception as e:
#             print(f"Error fetching image for {item['name']}: {e}")
#             continue

#     # Calculate grid dimensions
#     n_images = len(images)
#     grid_size = math.ceil(math.sqrt(n_images))
    
#     # Calculate individual cell size
#     cell_width = output_size[0] // grid_size
#     cell_height = (output_size[1] // grid_size) + 40  # 40 pixels for text
    
#     # Create blank white image
#     background = Image.new('RGB', 
#                          (cell_width * grid_size, cell_height * grid_size), 
#                          'white')
#     draw = ImageDraw.Draw(background)
    
#     try:
#         font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 14)
#     except:
#         font = ImageFont.load_default()

#     # Place each image and its title
#     for idx, (img, title) in enumerate(zip(images, titles)):
#         row = idx // grid_size
#         col = idx % grid_size
        
#         # Calculate available space for image
#         available_width = cell_width - 20  # 10px padding each side
#         available_height = cell_height - 50  # Space for text and padding
        
#         # Get original aspect ratio
#         orig_width, orig_height = img.size
#         aspect_ratio = orig_width / orig_height
        
#         # Calculate new dimensions maintaining aspect ratio
#         if aspect_ratio > 1:  # Wider than tall
#             new_width = available_width
#             new_height = int(new_width / aspect_ratio)
#             if new_height > available_height:
#                 new_height = available_height
#                 new_width = int(new_height * aspect_ratio)
#         else:  # Taller than wide
#             new_height = available_height
#             new_width = int(new_height * aspect_ratio)
#             if new_width > available_width:
#                 new_width = available_width
#                 new_height = int(new_width / aspect_ratio)
        
#         # Resize image
#         img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
#         # Calculate centering positions
#         x = col * cell_width + ((cell_width - new_width) // 2)
#         y = row * cell_height + 35 + ((available_height - new_height) // 2)
        
#         # Paste image
#         background.paste(img, (x, y))
        
#         # Add title
#         text_x = col * cell_width + (cell_width // 2)
#         text_y = row * cell_height + 10
#         draw.text((text_x, text_y), title, 
#                  fill='black', 
#                  font=font, 
#                  anchor="mm")

# #     return background
# def create_dsa_image_grid(gc, item_list, output_size=(2000, 1000)):
#     """
#     Creates a grid from DSA images with better spacing
#     """
#     images = []
#     titles = []
    
#     # Fetch all images and prepare titles
#     for item in item_list:
#         try:
#             img = fetch_dsa_image(gc, item['_id'])
#             images.append(img)
#             titles.append(item['name'].split('.')[0:2][0])
#         except Exception as e:
#             print(f"Error fetching image for {item['name']}: {e}")
#             continue

#     # Calculate grid dimensions - aim for a more rectangular layout
#     n_images = len(images)
#     n_cols = 2 # Max 5 images per row
#     n_rows = math.ceil(n_images / n_cols)
    
#     # Calculate individual cell size
#     cell_width = output_size[0] // n_cols
#     cell_height = output_size[1] // n_rows
    
#     # Create blank white image
#     background = Image.new('RGB', 
#                          (output_size[0], output_size[1]), 
#                          'white')
#     draw = ImageDraw.Draw(background)
    
#     try:
#         font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 14)
#     except:
#         font = ImageFont.load_default()

#     # Place each image and its title
#     for idx, (img, title) in enumerate(zip(images, titles)):
#         row = idx // n_cols
#         col = idx % n_cols
        
#         # Calculate available space for image
#         available_width = cell_width - 20  # 10px padding each side
#         available_height = cell_height - 30  # Space for text and padding
        
#         # Get original aspect ratio
#         orig_width, orig_height = img.size
#         aspect_ratio = orig_width / orig_height
        
#         # Calculate new dimensions maintaining aspect ratio
#         if aspect_ratio > 1:
#             new_width = available_width
#             new_height = int(new_width / aspect_ratio)
#             if new_height > available_height:
#                 new_height = available_height
#                 new_width = int(new_height * aspect_ratio)
#         else:
#             new_height = available_height
#             new_width = int(new_height * aspect_ratio)
#             if new_width > available_width:
#                 new_width = available_width
#                 new_height = int(new_width / aspect_ratio)
        
#         # Resize image
#         img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
#         # Calculate centering positions
#         x = col * cell_width + ((cell_width - new_width) // 2)
#         y = row * cell_height + 25 + ((available_height - new_height) // 2)
        
#         # Paste image
#         background.paste(img, (x, y))
        
#         # Add title
#         text_x = col * cell_width + (cell_width // 2)
#         text_y = row * cell_height + 10
#         draw.text((text_x, text_y), title, 
#                  fill='black', 
#                  font=font, 
#                  anchor="mm")

#     return background


# def create_dsa_image_grid(gc, item_list, output_size=(2000, 800)):
#     """
#     Creates a grid from DSA images with 2 rows
#     """
#     images = []
#     titles = []
    
#     # Fetch all images and prepare titles
#     for item in item_list:
#         try:
#             img = fetch_dsa_image(gc, item['_id'])
#             images.append(img)
#             titles.append(item['name'].split('.')[0:2][0])
#         except Exception as e:
#             print(f"Error fetching image for {item['name']}: {e}")
#             continue

#     # Force 2 rows, calculate columns needed
#     n_images = len(images)
#     n_rows = 2
#     n_cols = math.ceil(n_images / n_rows)  # This ensures we have enough columns
    
#     # Calculate individual cell size based on number of columns
#     cell_width = output_size[0] // n_cols
#     cell_height = (output_size[1] // n_rows)
    
#     # Create blank white image
#     background = Image.new('RGB', 
#                          (cell_width * n_cols, cell_height * n_rows), 
#                          'white')
#     draw = ImageDraw.Draw(background)
    
#     try:
#         font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 14)
#     except:
#         font = ImageFont.load_default()

#     # Place each image and its title
#     for idx, (img, title) in enumerate(zip(images, titles)):
#         # Calculate position in 2-row grid
#         row = idx % n_rows  # Alternates between 0 and 1
#         col = idx // n_rows  # Increments after every 2 items
        
#         # Calculate available space for image
#         available_width = cell_width - 20
#         available_height = cell_height - 30
        
#         # Get original aspect ratio
#         orig_width, orig_height = img.size
#         aspect_ratio = orig_width / orig_height
        
#         # Calculate new dimensions maintaining aspect ratio
#         if aspect_ratio > 1:
#             new_width = available_width
#             new_height = int(new_width / aspect_ratio)
#             if new_height > available_height:
#                 new_height = available_height
#                 new_width = int(new_height * aspect_ratio)
#         else:
#             new_height = available_height
#             new_width = int(new_height * aspect_ratio)
#             if new_width > available_width:
#                 new_width = available_width
#                 new_height = int(new_width / aspect_ratio)
        
#         # Calculate centering positions
#         x = col * cell_width + ((cell_width - new_width) // 2)
#         y = row * cell_height + 25 + ((available_height - new_height) // 2)
        
#         # Paste image
#         background.paste(img, (x, y))
        
#         # Add title
#         text_x = col * cell_width + (cell_width // 2)
#         text_y = row * cell_height + 10
#         draw.text((text_x, text_y), title, 
#                  fill='black', 
#                  font=font, 
#                  anchor="mm")


#     return background

def create_dsa_image_grid(gc, item_list, output_size=(2000, 800)):
    """
    Creates a grid from DSA images with 2 rows, flowing horizontally
    """
    images = []
    titles = []
    
    # Fetch all images and prepare titles
    for item in item_list:
        try:
            img = fetch_dsa_image(gc, item['_id'])
            images.append(img)
            titles.append(item['name'].split('.')[0:2][0])
        except Exception as e:
            print(f"Error fetching image for {item['name']}: {e}")
            continue

    # Force 2 rows, calculate columns needed
    n_images = len(images)
    n_cols = 2
    n_rows = math.ceil(n_images / n_rows)
    
    # Calculate individual cell size
    cell_width = output_size[0] // n_cols
    cell_height = (output_size[1] // n_rows)
    
    # Create blank white image
    background = Image.new('RGB', 
                         (cell_width * n_cols, cell_height * n_rows), 
                         'white')
    draw = ImageDraw.Draw(background)
    
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 14)
    except:
        font = ImageFont.load_default()

    # Place each image and its title
    for idx, (img, title) in enumerate(zip(images, titles)):
        # Calculate position - now flowing horizontally first
        row = idx // n_cols  # Changes row after filling a row of columns
        col = idx % n_cols   # Moves through columns first
        
        # Calculate available space for image
        available_width = cell_width - 20
        available_height = cell_height - 30
        
        # Get original aspect ratio
        orig_width, orig_height = img.size
        aspect_ratio = orig_width / orig_height
        
        # Calculate new dimensions maintaining aspect ratio
        if aspect_ratio > 1:
            new_width = available_width
            new_height = int(new_width / aspect_ratio)
            if new_height > available_height:
                new_height = available_height
                new_width = int(new_height * aspect_ratio)
        else:
            new_height = available_height
            new_width = int(new_height * aspect_ratio)
            if new_width > available_width:
                new_width = available_width
                new_height = int(new_width / aspect_ratio)
        
        # Calculate centering positions
        x = col * cell_width + ((cell_width - new_width) // 2)
        y = row * cell_height + 25 + ((available_height - new_height) // 2)
        
        # Paste image
        background.paste(img, (x, y))
        
        # Add title
        text_x = col * cell_width + (cell_width // 2)
        text_y = row * cell_height + 10
        draw.text((text_x, text_y), title, 
                 fill='black', 
                 font=font, 
                 anchor="mm")

    return background




In [43]:
## Get 10 items for now..
compositedItems = list(gc.listItem(compositeFolderId))


itemList = [gc.getItem(item['_id']) for item in compositedItems][:40]

grid_image = create_dsa_image_grid(gc, itemList)
grid_image.save('tma_grid_1to40.png')


# itemList = [gc.getItem(item['_id']) for item in compositedItems][40:80]

# grid_image = create_dsa_image_grid(gc, itemList)
# grid_image.save('tma_grid_41to80.png')

# itemList = [gc.getItem(item['_id']) for item in compositedItems][80:120]

# grid_image = create_dsa_image_grid(gc, itemList)
# grid_image.save('tma_grid_81to120.png')



UnboundLocalError: cannot access local variable 'n_rows' where it is not associated with a value